## Section3 精度向上Tips

### 3.1 Clippingによる勾配爆発への対処

SimpleRNNの課題として、勾配爆発と勾配消失があげられる。

このうち、勾配爆発については**Gradient Clipping**とよばれる、勾配の大きさそのものを制限ししまうという手法が有効。

![Clipping](https://github.com/reminaya
no/matsuo/blob/master/lesson3/figures/Clipping.png?raw=true)

この図のグリッド(格子状の線)が損失関数のパラメータに対する振る舞いを表している。勾配は各点でのグリッド面の傾きに対応している。

学習にあたっては、損失関数が最小となる点を探したい。こうした急峻な形状の損失関数では、「崖」に相当する部分で勾配爆発が発生する。

Clippingを行わない場合、極端な例では上図の左のように、「崖」での勾配爆発で探索点を一気に押し戻され、それまでの探索が無駄になってしまう。

そこで、Clippingを行うことで、上図右のように、課題な勾配については制限し、探索が過剰に変化することを抑制している。

ただし、あまりに制限値を小さくとると、常に勾配が小さくなり、学習のスピードが落ちる点には注意を要する。

kerasでGradient Clippingを行うには、Optimizerを指定する際に引数としてclipnormまたはclipvalueを指定する。

```python
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, clipnorm=1.)    # clipnormは勾配の2乗ノルムの最大値を制限する
sgd2 = optimizers.SGD(lr=0.01, clipvalue=1.)  # clipvalueは勾配の"要素"の絶対値の大きさを制限する
# clipnormの方が勾配の方向を変えないという利点があるが、経験的にはどちらの振る舞いも大差ない

ada = optimizers.Adagrad(lr=0.01, clipnorm=1.)# SGDに限らずすべてのoptimizerで指定可能

#model.compile(loss='mean_squared_error', optimizer=sgd)
```

なお、2乗ノルム $||$・$||_2$ とは、
- ベクトルに対する、通常の距離(ベクトル$v$に対して $||v||_2 = \sqrt{v_1^2+v_2^2+\cdots+v_n^2}$ )

- 行列に対する、Frobeniusノルム(行列$A$に対して $||A||_2 = \sqrt{\Sigma_{i=1}^m \Sigma_{j=1}^n a_{ij}^2 }$)

のことを表す。

### 3.2 ショートカットショートカットとゲートによる勾配消失への対応

Clippingは勾配爆発への対処として有力であるが、勾配消失への対処としては**ショートカット**とよばれる手法が効果的。

これは、各層の出力にその層への入力だったものも加えてしまうという手法。

(RNNに限らず、一般に第l層目の出力を $o^{(l)} = f^{(l)}(o^{(l-1)})+o^{(l-1)}$ とすることで可能)

![shortcut](https://github.com/reminayano/matsuo/blob/master/lesson3/figures/shortcut.png?raw=true)

一見、これが勾配消失に有効か疑わしいが、このショートカットにより、この層の勾配が **「１＋元の勾配」** と増加する。

そのため、勾配の積が積み重なる、入り口に近い層でも勾配が消失することなく、パラメータの更新が可能となる。

-----
また、このショートカットの概念に加え、**ゲート**とい概念も重要。

これは、ショートカットの一般化として重み付き和を考えるもの。

(つまり、$f^{(l)}(o^{(l-1)})$ と $o^{(l-1)}$ に各々係数を掛けたうえで足し合わせる。)

![gate](https://github.com/reminayano/matsuo/blob/master/lesson3/figures/gate.png?raw=true)

なお図中の $\odot$ は要素ごとの積(アダマール積)を表す。

この**ゲートの係数も学習**することにより、前の層から
の情報と現在の層による情報の重みを最適に調整できる。

つまり以前の層からの情報の忘却度合いをちょうどよく決められます。

----
これまでのテクニックはRNNに限らず、一般のDNNの層を重ねる際に使用できる。

RNNの場合は、特にこのゲートを(層方向だけでなく)時間方向に適用することで長期記憶や短期記憶を実現した、**ゲート付きRNN**が重要となる。

次節以降では、このゲート付きRNNとして代表的なLSTMとGRUについて触れる。

### 3.4 LSTM

**Long Short Term Memory(LSTM)** はゲートの考え方を時間方向の隠れ状態の計算に用いることで、系列内の長期的な相互依存性をモデル化できるようにしたRNNで、頻繁に使用されるモデルの1つ。

下図にLSTMの時点tでのモデルの模式図を示す。(このような素子が横に系列長Tだけ並んでいる)

![lstm](https://github.com/reminayano/matsuo/blob/master/lesson3/figures/lstm.png?raw=true)

LSTMでは時点tでの入力情報(時点tでの入力$x_t$、前時点での出力$h_{t-1}$ を結合したもの)を入力ゲート$i_t$ を介して取り込んでいる。

この入力情報を用いて、前時点までの長期的な系列情報 $c_{t-1}$ を$c_t$ に更新する。

なお、このとき前時点までの情報$c_{t-1}$ をどれだけ重視するか、忘却ゲート$f_t$ が制御している。

最終的な出力は、時点tまでの系列情報$c_t$ を出力ゲート$o_t$ によって調整することで決定される。

これらを線形変換や活性化関数を含めて数式で表現すると以下のようになる。

$$
\begin{eqnarray}
\bar{h}_t & = & \tanh(W_\bar{h}x_t+R_\bar{h}h_{t-1}+b_\bar{h}) \\
c_t & = & i_t \odot \bar{h}_t+f_t \odot c_{t-1} \\
h_t & = & o_t\odot \tanh (c_t)
\end{eqnarray}
$$

そて、各ゲートの係数 $i_t、f_t、o_t$ は以下の様にして、時点tの入力情報を元に決定される。

パラメータ $W、R、b$ を学習により決定することで、ゲート係数も多様なデータに柔軟に対応できる。

$$
\begin{eqnarray}
i_t & = & \sigma(W_ix_t+R_ih_{t-1}+b_i) \\
f_t & = & \sigma(W_fx_t+R_fh_{t-1}+b_f) \\
o_t & = & \sigma(W_ox_t+R_oh_{t-1}+b_o) \\
\end{eqnarray}
$$

ここまで詳しくLSTMの構造を見てきたが、kerasにおいてはそうした詳細を気にする必要がほとんどない。keras.layers.LSTMを用いるだけで利用できるので。

大事なのは、RNNのなかっでも長い系列に強い(系列内の長期的な相互依存性をモデル可能)モデルであるという特性を認識しておくこと。

引数
- units : ユニット数(系列長T)
- activation : 活性化関数
- recurrect_activation : ゲート係数の計算で使用する活性化関数
- use_bias : バイアスベクトル( $Wx_t+Rh_{t-1}$ に付け加えるベクトル)を使用するか
- {kernel, recurrent, bias}_initializer : 各パラメータの初期化法(kernelはW、reccurentはRを指す)
- unit_forget_bias : 忘却ゲートを1に初期化
- {kernel, recurrent, bias, activity}_regularizer : 各パラメータの正規化(activityは出力=activationを指す)
- {kernel, recurrrent, bias}_constraint : 各パラメータに課す制約
- dropout : 入力についてのdropoutの比率(Wに対するdropout)
- recurrent_dropout : 再帰についてのdropoutの比率(Rに対するdropout)
- return_sequences : Falseなら出力としては系列の最後の出力のみ($o_T$のみ)を返す、Trueなら出力として完全な出力($o_1, o_2, \cdots, o_T$)を返す
- return_state : Trueのときは出力とともに最後の状態($S_T$)を返す
- go_backwards : Trueのときは入力系列を後ろから処理する(出力も逆順に)
- stateful : Trueのときは、前バッチの各サンプルに対する最後の状態を、次のバッチのサンプルに対する初期状態として引き継ぐ
- unroll : (高速化のためのオプション) Trueのときは再帰が展開され高速化されるが、よりメモリに負荷がかかる。(短い系列にのみ適する)

```python
keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True,
                  kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                  unit_forget_bias=True,
                  kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                  kernel_constraint=None, recurrent_constraint=None, bias_constraint=None,
                  dropout=0.0, recurrent_dropout=0.0, implementation=1,
                  return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)
```

### 3.3 GRU

**Gated Recurrent Unit(GRU)** は、ゲートの考え方を利用しながら、隠れ状態ベクトル$h_t$のみに長期の情報も集約したモデル。

(LSTMでは、長期の状態と短期の状態を$c_t, h_t$ の2本で管理していた)

![gru](https://github.com/reminayano/matsuo/blob/master/lesson3/figures/gru.png?raw=true)

このように、状態ベクトルは$h_t$のみ、ゲートも$r_t, z_t$ の2つのみで、LSTMと比較して(同じ系列長なら)計算量も使用空間量も少なく済む。

(ただし、RNNとしての性能については、タスクごとにLSTMとGRUのどちらが有意となるかは違っていて、甲乙つけがたい)

GRUの実装もkerasでは簡単に行うことができ、次のkeras.layers.GRUを用いる。

主な引数
- units : ユニット数(系列長T)
- activation : 活性化関数
- recurrect_activation : 内部で使用する活性化関数
- use_bias : バイアスベクトル( $Ux_t+Wh_{t-1}$ に付け加えるベクトル)を使用するか
- {kernel, recurrent, bias}_initializer : 各パラメータの初期化法(kernelはW、reccurentはRを指す)
- unit_forget_bias : 忘却ゲートを1に初期化
- {kernel, recurrent, bias, activity}_regularizer : 各パラメータの正規化(activityは出力=activationを指す)
- {kernel, recurrrent, bias}_constraint : 各パラメータに課す制約
- dropout : 入力についてのdropoutの比率(Wに対するdropout)
- recurrent_dropout : 再帰についてのdropoutの比率(Rに対するdropout)
- return_sequences : Falseなら出力としては系列の最後の出力のみ($o_T$のみ)を返す、Trueなら出力として完全な出力($o_1, o_2, \cdots, o_T$)を返す
- return_state : Trueのときは出力とともに最後の状態($S_T$)を返す
- go_backwards : Trueのときは入力系列を後ろから処理する(出力も逆順に)
- stateful : Trueのときは、前バッチの各サンプルに対する最後の状態を、次のバッチのサンプルに対する初期状態として引き継ぐ
- unroll : (高速化のためのオプション) Trueのときは再帰が展開され高速化されるが、よりメモリに負荷がかかる。(短い系列にのみ適する)

```python
keras.layers.GRU(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                 kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                 kernel_constraint=None, recurrent_constraint=None, bias_constraint=None,
                 dropout=0.0, recurrent_dropout=0.0, implementation=1,
                 return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False, reset_after=False)
```
